In [4]:
import cv2 ##Para instalar esta librería: conda install opencv
import time
import os
import pandas as pd
import tarfile as tar

In [5]:
def video_to_frames(input_loc, output_loc, resize_w = 0, resize_h = 0, s = 1, gray = False):
    """Funcion para extraer los frames de un video
    y guardarlos en un directorio dado comprimidos como tar gz.
    Argumentos:
        input_loc: directorio de los videos.
        output_loc: directorio donde se quieren guardar los frames.
        resize_w: tamaño en pixeles que se quieren reshapear el ancho de la imagen.
        resize_h: tamaño en pixeles que se quieren reshapear el alto de la imagen.
    Si omito uno de los dos anteriores parámetros la imagen será proporcional a la imagen anterior.
        s: cada cuantos segundos se quiere guardar una imágen. Si es cero trae todos los fotogramas del video.
        gray: si se quieren guardar imagenes en escala de grices.
    Devuelve:
        Un archivo csv con el resumen de los videos procesados
    """
    
    ## Si el directorio de salida no esta creado, lo creo
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    
    ## Guardo los nombres de los archivos que estan en el directorio de entrada
    videos = os.listdir(input_loc)
    #Remuevo este archivo oculto (sólo puede aparecer en macOS)
    try:
        videos.remove('.DS_Store')
    except ValueError:
        pass

    ## Si no hay archivos en el directorio me lo indica y no hace nada
    if len(videos) == 0:
        print('No hay archivos en el directorio')
    else:
        info_videos = pd.DataFrame()
        # Tiempo de inicio global
        time_start_g = time.time()
        
        for i in videos:
            # Tiempo de inicio
            time_start = time.time()
            # Capturo el video
            cap = cv2.VideoCapture(input_loc + i)
            
            print ("Convirtiendo el video número %d " %(videos.index(i)+1) + "de un total de %d " %len(videos) + "videos.")
            print("Transformando el video " + i + "\n")
            # Extraigo el numero de frames y lo imprimo
            fps = cap.get(cv2.CAP_PROP_FPS)
            frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frameCount/fps
            video_length = int(frameCount) - 1
            alto_frame = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            ancho_frame = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            
            print("Info del video")
            print('fps = ' + str(fps))
            print('Cantidad de frames = ' + str(frameCount))
            print('Dimensiones_frame = '+str(ancho_frame)+"x"+ str(alto_frame))
            print('Duración (S) = ' + str(duration))
            minutes = int(duration/60)
            seconds = round(duration%60,2)
            print('Duración (M:S) = ' + str(minutes) + ':' + str(seconds) + '\n')
            
            
            count_frames = 0
            sec_video = 0
            count_inner = 0
            # Convierto el video en imágenes
            while cap.isOpened():
                # Extraigo el frame
                ret, frame = cap.read()
                
                # Modifico los frame, resize, escala de grises
                if resize_w != 0 and resize_h != 0:
                    frame = cv2.resize(frame,(resize_w, resize_h), interpolation = cv2.INTER_AREA)
                elif resize_w != 0 and resize_h == 0: 
                    frame = cv2.resize(frame,(resize_w, int(resize_w*alto_frame/ancho_frame)), interpolation = cv2.INTER_AREA)
                elif resize_w == 0 and resize_h != 0:
                    frame = cv2.resize(frame,(int(resize_h*ancho_frame/alto_frame),resize_h), interpolation = cv2.INTER_AREA)
                    
                if gray == True:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
                
                
                count_frames = count_frames + 1
                count_inner = count_inner + 1
                
                if s == 0:
                    if count_frames%int(round(fps,0)) == 0:
                        sec_video = sec_video + 1
                        count_inner = 1
                        sec_video = sec_video + 1
                        count_inner = 1
                        # Escribo el frame como jpg.
                        cv2.imwrite(output_loc + "/"+ i.rsplit('.', 1)[0] +"_%dseg" %sec_video +"_%d" %count_inner +"_%#05d.jpg" % (count_frames), frame)
                elif count_frames%(int(round(fps,0))*s) == 0:
                    sec_video = sec_video + 1*s
                    # Escribo el frame como jpg cada s segundos.
                    cv2.imwrite(output_loc + "/"+ i.rsplit('.', 1)[0] +"_%dseg" %sec_video +"_%#05d.jpg" % (count_frames), frame)
                
                if count_frames%(10**(len(str(video_length))-1)) == 0:
                    # Imprimo estadísticas parciales
                    print ("Se extrajo %d frames" % count_frames + " de un total de %d frames" %video_length)
                            
                # Si no hay más frames
                if (count_frames > (video_length)):
                    # Tiempo final
                    time_end = time.time()
                    # Cierro el video
                    cap.release()
                    # Imprimo estadísticas finales
                    print ("Se terminó la extracción de los frames.\n%d frames extraidos" % count_frames)
                    print ("Tardó %d seconds para la conversión." % (time_end-time_start))
                    print("-----------------------------------------\n")
                    
                    #Creo un diccionario con la info del video
                    dic = {'Nombre_archivo': i, 
                           'fps':round(fps,3), 
                           'Cant_frames': frameCount,
                           'Dimensiones_frame': str(ancho_frame)+"x"+ str(alto_frame),
                           'Duracion_s': round(duration,3),
                           'Duracion_M_S': str(minutes) + ':' + str(seconds),
                           'Tiempo_proceso_M': (time_end-time_start)/60
                          }
                    #Convierto el diccionario en data.frame
                    data = pd.DataFrame(data=[dic], columns = ['Nombre_archivo', 'fps', 'Cant_frames', 
                                                               'Dimensiones_frame', 'Duracion_s', 
                                                               'Duracion_M_S', 'Tiempo_proceso_M'])
                    #Agrego la info de los videos ya procesados
                    info_videos = info_videos.append(data)
                    break
            
            ### Comprimo las imagenes
            #Listo las imagenes recien creadas
            imagenes = os.listdir(output_loc)
            #Solo me quedo con los nombres de los archivos que terminan en jpg
            imagenes = [imagen for imagen in imagenes if  imagen.endswith('.jpg')]
            
            #Comprimo las imagenes
            with tar.open(output_loc+ i.rsplit('.', 1)[0] + '.tar', 'w:gz') as tarf:
                for name in imagenes:
                    tarf.add(output_loc + name)
                tarf.close()
            
            ###Borro las imagenes
            [os.remove(output_loc + file) for file in imagenes if os.path.isfile(output_loc + file)]
            
        info_videos.to_csv('info_videos.csv', sep =';', encoding='utf-8', index=False)
        
        # Tiempo de finalizacion global
        time_end_g = time.time()
        print('El proceso global de conversión tardó %d' %((time_end_g - time_start_g)/60) +'min')
        
        cap.release()

In [7]:
#video_to_frames('.\\videos\\', '.\\imagenes\\200x200\\',resize_w = 200, resize_h = 200, gray = True)
#video_to_frames('./Video/', './Imagenes/', resize_w = 200, gray = True)
video_to_frames('../Videos_Buses/Nuevos/', './Imagenes/', resize_w = 200, gray = True, s = 2)

Convirtiendo el video número 1 de un total de 6 videos.
Transformando el video L33_Atras_Int202_20190424_170001.mp4

Info del video
fps = 21.010955496357266
Cantidad de frames = 67349
Dimensiones_frame = 1280.0x720.0
Duración (S) = 3205.4230000000002
Duración (M:S) = 53:25.42

Se extrajo 10000 frames de un total de 67348 frames
Se extrajo 20000 frames de un total de 67348 frames
Se extrajo 30000 frames de un total de 67348 frames
Se extrajo 40000 frames de un total de 67348 frames
Se extrajo 50000 frames de un total de 67348 frames
Se extrajo 60000 frames de un total de 67348 frames
Se terminó la extracción de los frames.
67349 frames extraidos
Tardó 399 seconds para la conversión.
-----------------------------------------

Convirtiendo el video número 2 de un total de 6 videos.
Transformando el video L33_Frente_Int228_20190424_165959.mp4

Info del video
fps = 20.00287739252689
Cantidad de frames = 72020
Dimensiones_frame = 1280.0x720.0
Duración (S) = 3600.482
Duración (M:S) = 60:0.48
